In [14]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
config['augmentations']['pad_trunc_noise_multiplier']


'0.025'

In [28]:
from transforms import getTransforms

transformParams = getTransforms(True)
print(transformParams)
# for transform in transformParams:
#     print(transform)


[{'before_cochannel': [<audiomentations.augmentations.gain.Gain object at 0x000001DCD3E22A10>, <audiomentations.augmentations.time_stretch.TimeStretch object at 0x000001DCD51EA9E0>, <audiomentations.augmentations.shift.Shift object at 0x000001DCD51EA4A0>], 'audio': [<audiomentations.augmentations.add_gaussian_noise.AddGaussianNoise object at 0x000001DCD51E92D0>, <audiomentations.augmentations.pitch_shift.PitchShift object at 0x000001DCD51EA3E0>]}]


In [ ]:
import pyaudio
import wave
# from torchaudio.io import StreamReader

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 8000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    print(data)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


In [ ]:
import Augmentation
import torch
from AudioDataset import transformData
from torch.utils.data import Dataset
import audiomentations
from transforms import getTransforms

audio_paths = Augmentation.getAudioPaths('./data')

# audio_paths += Augmentation.getAudio(
#     'E:/Processed Singapore Speech Corpus/ENV')[0:1000]


audio_train_dataset = transformData(audio_paths, transformParams=getTransforms(True), generateCochannel=True)

test_dataloader = torch.utils.data.DataLoader(
    audio_train_dataset,
    batch_size=64,
    num_workers=0,
    shuffle=True,
)
   

batch = next(iter(test_dataloader))
batch

In [ ]:
import matplotlib.pyplot as plt


# batch = next(iter(test_dataloader))

idx = 0
print(batch[1][idx].item())
plt.imshow(batch[0][idx][0])
plt.colorbar()
plt.show()


